In [ ]:
from core.utils import Tibanna
from core import ff_utils
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

all_sets = ['dciclab:rao_rep07',
'dciclab:rao_rep02',
'dciclab:rao_rep12',
'dciclab:rao_rep13',
'dcic:Selvaraj_gm12878_hic',
'dcic:Jin_imr90_hic']

env = 'fourfront-webdev'
tibanna = Tibanna(env=env)

ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

for my_rep_set in all_sets:
    rep_resp = ff_utils.get_metadata(my_rep_set, connection=ff)['experiments_in_set']
    print my_rep_set
    my_aliases = []
    my_files = []
    for exp in rep_resp:
        exp_resp = ff_utils.get_metadata(exp, connection=ff)
        exp_files = exp_resp['files']
        al = exp_resp['aliases'][0].split(':')[1].upper()
         
        used_file = False
        for fastq_file in exp_files:
            file_resp = ff_utils.get_metadata(fastq_file, connection=ff)  
            if file_resp.get('instrument', 'Illumina my seq') != 'Illumina MiSeq':
                used_file = True
                f_al = file_resp['aliases'][0]
                if f_al.startswith('dcic:H'):
                    my_files.append(f_al.split('_')[1].upper())
                else:
                    my_files.append(f_al.split('_')[2].upper())
       
    
        if used_file:
            my_aliases.append(al)
    print ', '.join(my_aliases)
    print len(list(sorted(set(my_files))))
    print ', '.join(list(sorted(set(my_files))))
    print
